In [91]:
# please create a program hich asks user to inout a name of song check if the song is in hot100 billboard_df
# if yes recommend another song  if no check in rock_df yes or no

In [1]:
import config

In [2]:
import csv
import pandas as pd
import numpy as np
import random
import pickle
import spotipy
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

hot_100_df = pd.read_csv("hot_100_df1.csv")


# top_28_rock_df = pd.read_csv("top28_rock_songs_df.csv")

In [3]:
with open('kmeans_model.pickle', 'rb') as file:
    kmeans_model = pickle.load(file)

In [4]:
final_df = pd.read_csv('final_df.csv')

In [5]:
kmeans_model

KMeans(n_clusters=6, random_state=1234)

In [6]:
final_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,id
0,0.520319,0.403121,0.155294,0.816787,0.694184,-0.506695,-0.290501,-0.349655,-0.502811,-0.387373,0.276160,-0.128405,0.163282,1,1GEBsLDvJGw7kviySRI6GX
1,-1.201826,-1.294036,0.975181,-0.767962,0.694184,-0.592775,-0.197642,-0.299393,-0.678863,-0.072418,-2.526637,0.528527,0.163282,3,3fqwjXwUGN6vbzIwvyFMhx
2,0.111675,0.385018,-0.937888,0.930194,0.694184,-0.578960,-0.452145,-0.349738,-0.708713,-0.915024,-0.102684,-0.083871,0.163282,5,1D4PL9B8gOg78jiHg3FvBb
3,0.053297,-0.542761,1.248476,0.111309,0.694184,-0.585336,0.424860,-0.349738,-0.557637,-0.391463,0.897792,-0.206445,0.163282,5,10M2Ex445zw585Ducldzkw
4,0.461942,-0.452246,0.975181,0.298356,0.694184,-0.584274,0.001834,-0.349738,-0.539361,-0.035605,-1.447301,-0.693411,0.163282,1,1HXy5I3HTWq8OvxCn0z7G7


In [7]:
hot_100 = {song.lower(): {'song': song, 'artist': artist} for song, artist in zip(hot_100_df['song'], hot_100_df['artist'])}

In [8]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [ ]:
user_song = input("Please input name of a song: ").lower()

In [ ]:
scaler = StandardScaler()

if user_song in hot_100:
    suggest_songs = [song for song in hot_100 if song != user_song]

if suggest_songs:
    random_song = random.choice(suggest_songs)
    print(f"{hot_100[user_song]['song']} by {hot_100[user_song]['artist']} is on the Hot 100 list.\n\nYou may also like this other Hot 100 song:\n\n{hot_100[random_song]['song']} by {hot_100[random_song]['artist']}")
    
else:
    results = sp.search(q=user_song, type='track', limit=1)
    
    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
        audio_features = sp.audio_features([track_id])
        
    else:
        print("Song not found.")
    
    df = pd.DataFrame(audio_features)
    reqd_features = df.select_dtypes(include='number')
    scaled_features = scaler.fit_transform(reqd_features)
    pred_cluster = kmeans_model.predict(scaled_features)
    
    random_id = np.random.choice(final_df['id'].values)
    
    random_track = sp.track(random_id)
    
    print(f"Your inputted song is not on the Hot 100 list.\n\nYou may however like {random_track['name']} by {random_track['artists'][0]['name']}\n\n")
    
    print("The original song you had inputted:\n")
    display(play_song(track_id))
    
    print("The song recommended by our system:\n")
    display(play_song(random_id))